# ⏳ Warteschlangentheorie in der Produktionsplanung

**Intelligente Produktionsplanung und KI**  
Universität Stuttgart  
Master Maschinenbau / Technologiemanagement

---

## Lernziele

Nach Bearbeitung dieses Notebooks können Sie:

1. **Warteschlangensysteme** identifizieren und mit der Kendall-Notation beschreiben
2. Das **M/M/1-Modell** und **M/M/c-Modell** anwenden und interpretieren
3. **Kennzahlen** wie Auslastung, Wartezeit und Warteschlangenlänge berechnen
4. Eine **Kostenoptimierung** für Servicesysteme durchführen
5. **Simulationen** zur Validierung analytischer Modelle einsetzen

---

## 1. Einführung und Grundlagen

### Was ist Warteschlangentheorie?

Die **Warteschlangentheorie** (englisch: Queueing Theory) ist ein Teilgebiet der Wahrscheinlichkeitstheorie, das sich mit der mathematischen Analyse von Warteschlangen beschäftigt.

### Typische Anwendungen in der Produktion

| Anwendung | Kunden | Server |
|-----------|--------|--------|
| Werkzeugausgabe | Arbeiter | Ausgabestellen |
| Qualitätsprüfung | Werkstücke | Prüfstationen |
| Maschinenbearbeitung | Aufträge | CNC-Maschinen |
| Instandhaltung | Defekte Maschinen | Techniker |
| Materialbereitstellung | Produktionsaufträge | Gabelstapler |

### Kendall-Notation

Warteschlangensysteme werden mit der **Kendall-Notation** beschrieben: **A/B/c/K/N/D**

| Symbol | Bedeutung | Häufige Werte |
|--------|-----------|---------------|
| A | Ankunftsprozess | M (Poisson), D (deterministisch), G (allgemein) |
| B | Bedienprozess | M (exponentiell), D (deterministisch), G (allgemein) |
| c | Anzahl Server | 1, 2, 3, ... |
| K | Systemkapazität | ∞ (unbegrenzt) - oft weggelassen |
| N | Kundenpopulation | ∞ (unbegrenzt) - oft weggelassen |
| D | Warteschlangendisziplin | FIFO (First In First Out) - oft weggelassen |

**Beispiel:** M/M/1 bedeutet:
- **M**: Poisson-verteilte Ankünfte (Markov)
- **M**: Exponentiell verteilte Bedienzeiten (Markov)
- **1**: Ein Server

In [ ]:
# ===================================================================
# Imports und Konfiguration
# ===================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
from collections import deque
from math import factorial
import warnings
warnings.filterwarnings('ignore')

# Matplotlib Konfiguration
plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['font.size'] = 11

print("✅ Alle Bibliotheken erfolgreich geladen!")
print("\n📦 Verwendete Pakete:")
print(f"   - NumPy: {np.__version__}")
print(f"   - Pandas: {pd.__version__}")

---

## 2. Fallbeispiel: Werkzeugausgabe bei MechTech GmbH

### Problemstellung

Die **MechTech GmbH** betreibt eine Werkzeugausgabe, an der Mitarbeiter Werkzeuge und Material abholen. 

**Aktuelle Situation:**
- Arbeiter kommen unregelmäßig (Poisson-verteilt)
- Ausgabezeit variiert je nach Werkzeug (exponentiell verteilt)
- Aktuell: **1 Ausgabestelle**
- Beschwerden über lange Wartezeiten

**Ziel:** Analyse der aktuellen Situation und Optimierung der Anzahl Ausgabestellen.

In [ ]:
# ===================================================================
# Problemdaten definieren
# ===================================================================

print("🔧 FALLBEISPIEL: Werkzeugausgabe bei MechTech GmbH")
print("=" * 60)

# Systemparameter
lambda_rate = 15  # Ankunftsrate: 15 Arbeiter pro Stunde
mu_rate = 20      # Bedienrate: 20 Arbeiter pro Stunde (pro Server)

print("\n📊 SYSTEMPARAMETER")
print("-" * 40)
print(f"   Ankunftsrate (λ):     {lambda_rate} Arbeiter/Stunde")
print(f"   Bedienrate (μ):       {mu_rate} Arbeiter/Stunde")
print(f"   Mittlere Zwischenankunftszeit: {60/lambda_rate:.1f} Minuten")
print(f"   Mittlere Bedienzeit:           {60/mu_rate:.1f} Minuten")

# Auslastung berechnen
rho = lambda_rate / mu_rate
print(f"\n   Auslastung (ρ = λ/μ): {rho:.2f} = {rho*100:.0f}%")

if rho < 1:
    print("   ✅ System ist stabil (ρ < 1)")
else:
    print("   ❌ System ist INSTABIL (ρ ≥ 1) - Warteschlange wächst unbegrenzt!")

---

## 3. Das M/M/1-Modell

### Voraussetzungen

- Ankünfte folgen einem **Poisson-Prozess** mit Rate λ
- Bedienzeiten sind **exponentialverteilt** mit Rate μ
- **Ein Server** (c = 1)
- Unbegrenzte Warteschlange (FIFO)
- Stabilitätsbedingung: **ρ = λ/μ < 1**

### Wichtige Formeln

| Kennzahl | Formel | Bedeutung |
|----------|--------|----------|
| Auslastung | $\rho = \frac{\lambda}{\mu}$ | Anteil der Zeit, in der Server beschäftigt |
| Leerwahrscheinlichkeit | $P_0 = 1 - \rho$ | Wahrscheinlichkeit, dass System leer |
| Mittlere Anzahl im System | $L = \frac{\rho}{1-\rho}$ | Wartende + in Bedienung |
| Mittlere Anzahl in Warteschlange | $L_q = \frac{\rho^2}{1-\rho}$ | Nur Wartende |
| Mittlere Systemzeit | $W = \frac{1}{\mu - \lambda}$ | Warten + Bedienung |
| Mittlere Wartezeit | $W_q = \frac{\rho}{\mu - \lambda}$ | Nur Wartezeit |

In [ ]:
# ===================================================================
# M/M/1 Modell Implementation
# ===================================================================

def mm1_modell(lambda_rate, mu_rate):
    """
    Berechnet alle Kennzahlen für ein M/M/1-Warteschlangensystem.
    
    Parameter:
    ----------
    lambda_rate : float
        Ankunftsrate (Kunden pro Zeiteinheit)
    mu_rate : float
        Bedienrate (Kunden pro Zeiteinheit)
        
    Returns:
    --------
    dict : Dictionary mit allen Kennzahlen
    """
    
    # Stabilitätsprüfung
    if lambda_rate >= mu_rate:
        return {
            'Fehler': 'System ist instabil (λ ≥ μ)',
            'rho': lambda_rate / mu_rate
        }
    
    # Auslastung
    rho = lambda_rate / mu_rate
    
    # Kennzahlen berechnen
    P0 = 1 - rho                           # Leerwahrscheinlichkeit
    L = rho / (1 - rho)                    # Mittlere Anzahl im System
    Lq = (rho ** 2) / (1 - rho)            # Mittlere Anzahl in Warteschlange
    W = 1 / (mu_rate - lambda_rate)        # Mittlere Systemzeit
    Wq = rho / (mu_rate - lambda_rate)     # Mittlere Wartezeit
    
    return {
        'rho': rho,
        'P0': P0,
        'L': L,
        'Lq': Lq,
        'W': W,
        'Wq': Wq,
        'lambda': lambda_rate,
        'mu': mu_rate
    }

# Berechnung für aktuelle Situation
ergebnis_mm1 = mm1_modell(lambda_rate, mu_rate)

print("📊 M/M/1 ANALYSE - Werkzeugausgabe (1 Server)")
print("=" * 60)

if 'Fehler' not in ergebnis_mm1:
    print("\n📈 KENNZAHLEN:")
    print("-" * 45)
    print(f"   Auslastung (ρ):                    {ergebnis_mm1['rho']:.2%}")
    print(f"   Leerwahrscheinlichkeit (P₀):       {ergebnis_mm1['P0']:.2%}")
    print(f"   Mittlere Anzahl im System (L):     {ergebnis_mm1['L']:.2f} Arbeiter")
    print(f"   Mittlere Anzahl wartend (Lq):      {ergebnis_mm1['Lq']:.2f} Arbeiter")
    print(f"   Mittlere Systemzeit (W):           {ergebnis_mm1['W']*60:.1f} Minuten")
    print(f"   Mittlere Wartezeit (Wq):           {ergebnis_mm1['Wq']*60:.1f} Minuten")
    
    print("\n💡 INTERPRETATION:")
    print(f"   • Der Server ist {ergebnis_mm1['rho']*100:.0f}% der Zeit beschäftigt")
    print(f"   • Im Durchschnitt warten {ergebnis_mm1['Lq']:.1f} Arbeiter in der Schlange")
    print(f"   • Ein Arbeiter wartet durchschnittlich {ergebnis_mm1['Wq']*60:.0f} Minuten")
    print(f"   • Die gesamte Abwicklung dauert {ergebnis_mm1['W']*60:.0f} Minuten")
else:
    print(f"\n❌ {ergebnis_mm1['Fehler']}")

In [ ]:
# ===================================================================
# Visualisierung: Wahrscheinlichkeitsverteilung der Systemzustände
# ===================================================================

def plot_zustandswahrscheinlichkeiten(rho, max_n=15):
    """
    Visualisiert die Wahrscheinlichkeitsverteilung für n Kunden im System.
    
    Für M/M/1: P(n) = (1-ρ) * ρ^n (geometrische Verteilung)
    """
    
    n_werte = np.arange(0, max_n + 1)
    P_n = (1 - rho) * (rho ** n_werte)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # Balkendiagramm der Wahrscheinlichkeiten
    colors = ['green' if n == 0 else 'steelblue' for n in n_werte]
    ax1.bar(n_werte, P_n, color=colors, edgecolor='black', alpha=0.7)
    ax1.set_xlabel('Anzahl Kunden im System (n)', fontsize=11)
    ax1.set_ylabel('Wahrscheinlichkeit P(n)', fontsize=11)
    ax1.set_title(f'Zustandswahrscheinlichkeiten M/M/1 (ρ = {rho:.2f})', fontsize=12, fontweight='bold')
    ax1.set_xticks(n_werte)
    ax1.grid(axis='y', alpha=0.3)
    
    # Annotation für P(0)
    ax1.annotate(f'P(0) = {P_n[0]:.1%}\n(System leer)', 
                xy=(0, P_n[0]), xytext=(3, P_n[0]),
                fontsize=10, ha='left',
                arrowprops=dict(arrowstyle='->', color='green'))
    
    # Kumulative Verteilung
    P_kum = np.cumsum(P_n)
    ax2.plot(n_werte, P_kum, 'b-o', linewidth=2, markersize=6)
    ax2.axhline(y=0.9, color='r', linestyle='--', label='90%-Quantil')
    ax2.axhline(y=0.95, color='orange', linestyle='--', label='95%-Quantil')
    ax2.set_xlabel('Anzahl Kunden im System (n)', fontsize=11)
    ax2.set_ylabel('Kumulative Wahrscheinlichkeit P(N ≤ n)', fontsize=11)
    ax2.set_title('Kumulative Verteilungsfunktion', fontsize=12, fontweight='bold')
    ax2.set_xticks(n_werte)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Quantile berechnen
    for quantil in [0.9, 0.95]:
        n_quantil = np.argmax(P_kum >= quantil)
        print(f"   {quantil*100:.0f}%-Quantil: Mit {quantil*100:.0f}% Wahrscheinlichkeit sind max. {n_quantil} Kunden im System")

print("📊 ZUSTANDSWAHRSCHEINLICHKEITEN")
print("=" * 50)
plot_zustandswahrscheinlichkeiten(ergebnis_mm1['rho'])

---

## 4. Das M/M/c-Modell (Mehrere Server)

### Erweiterung auf c Server

Wenn wir **mehrere parallele Server** haben (z.B. mehrere Ausgabestellen), verwenden wir das **M/M/c-Modell**.

### Stabilitätsbedingung

$$\rho = \frac{\lambda}{c \cdot \mu} < 1$$

### Formeln (komplexer als M/M/1)

Die Wahrscheinlichkeit für ein leeres System $P_0$ wird benötigt:

$$P_0 = \left[ \sum_{n=0}^{c-1} \frac{(\lambda/\mu)^n}{n!} + \frac{(\lambda/\mu)^c}{c!} \cdot \frac{1}{1 - \lambda/(c\mu)} \right]^{-1}$$

In [ ]:
# ===================================================================
# M/M/c Modell Implementation
# ===================================================================

def mmc_modell(lambda_rate, mu_rate, c):
    """
    Berechnet alle Kennzahlen für ein M/M/c-Warteschlangensystem.
    
    Parameter:
    ----------
    lambda_rate : float
        Ankunftsrate (Kunden pro Zeiteinheit)
    mu_rate : float
        Bedienrate pro Server (Kunden pro Zeiteinheit)
    c : int
        Anzahl der Server
        
    Returns:
    --------
    dict : Dictionary mit allen Kennzahlen
    """
    
    # Hilfsgrößen
    r = lambda_rate / mu_rate  # Verkehrsintensität
    rho = r / c                 # Auslastung pro Server
    
    # Stabilitätsprüfung
    if rho >= 1:
        return {
            'Fehler': f'System ist instabil (ρ = {rho:.2f} ≥ 1)',
            'rho': rho,
            'c': c
        }
    
    # P0 berechnen (Wahrscheinlichkeit für leeres System)
    summe1 = sum([(r ** n) / factorial(n) for n in range(c)])
    summe2 = (r ** c) / (factorial(c) * (1 - rho))
    P0 = 1 / (summe1 + summe2)
    
    # Erlang-C Formel: Wahrscheinlichkeit zu warten
    P_warten = ((r ** c) / factorial(c)) * (1 / (1 - rho)) * P0
    
    # Kennzahlen
    Lq = P_warten * rho / (1 - rho)        # Mittlere Anzahl in Warteschlange
    L = Lq + r                              # Mittlere Anzahl im System
    Wq = Lq / lambda_rate                   # Mittlere Wartezeit
    W = Wq + 1 / mu_rate                    # Mittlere Systemzeit
    
    return {
        'c': c,
        'rho': rho,
        'P0': P0,
        'P_warten': P_warten,
        'L': L,
        'Lq': Lq,
        'W': W,
        'Wq': Wq,
        'lambda': lambda_rate,
        'mu': mu_rate
    }

# Vergleich verschiedener Serveranzahlen
print("📊 VERGLEICH: 1, 2 und 3 AUSGABESTELLEN")
print("=" * 65)

ergebnisse = {}
for c in [1, 2, 3]:
    if c == 1:
        ergebnisse[c] = mm1_modell(lambda_rate, mu_rate)
    else:
        ergebnisse[c] = mmc_modell(lambda_rate, mu_rate, c)

# Ergebnistabelle erstellen
vergleich_data = []
for c, erg in ergebnisse.items():
    if 'Fehler' not in erg:
        vergleich_data.append({
            'Server': c,
            'Auslastung (%)': f"{erg['rho']*100:.1f}",
            'Wartende (Lq)': f"{erg['Lq']:.2f}",
            'Im System (L)': f"{erg['L']:.2f}",
            'Wartezeit (min)': f"{erg['Wq']*60:.1f}",
            'Systemzeit (min)': f"{erg['W']*60:.1f}"
        })

df_vergleich = pd.DataFrame(vergleich_data)
print("\n" + df_vergleich.to_string(index=False))

print("\n💡 ERKENNTNISSE:")
print(f"   • Mit 2 Servern sinkt die Wartezeit von {ergebnisse[1]['Wq']*60:.0f} auf {ergebnisse[2]['Wq']*60:.1f} Minuten!")
print(f"   • Die Auslastung pro Server sinkt von {ergebnisse[1]['rho']*100:.0f}% auf {ergebnisse[2]['rho']*100:.0f}%")
print(f"   • Ein dritter Server bringt nur noch marginale Verbesserung")

---

## 5. Kostenoptimierung

### Das Trade-off Problem

Bei der Dimensionierung von Servicesystemen gibt es einen **Trade-off** zwischen:

- **Serverkosten**: Mehr Server = höhere Kosten
- **Wartekosten**: Mehr Server = kürzere Wartezeiten = geringere Kosten

### Gesamtkosten

$$K_{gesamt} = K_{Server} + K_{Warten} = c \cdot k_s + L \cdot k_w$$

wobei:
- $c$ = Anzahl Server
- $k_s$ = Kosten pro Server und Zeiteinheit
- $L$ = Mittlere Anzahl Kunden im System
- $k_w$ = Wartekosten pro Kunde und Zeiteinheit

In [ ]:
# ===================================================================
# Kostenanalyse
# ===================================================================

# Kostenparameter
kosten_server = 25      # €/Stunde pro Ausgabestelle (Personal + Betrieb)
kosten_warten = 45      # €/Stunde pro wartenden Arbeiter (Produktivitätsverlust)

print("💰 KOSTENANALYSE")
print("=" * 65)
print("\n📋 KOSTENANNAHMEN:")
print("-" * 40)
print(f"   Kosten pro Ausgabestelle:     {kosten_server} €/Stunde")
print(f"   Kosten pro wartenden Arbeiter: {kosten_warten} €/Stunde")
print("   (Opportunitätskosten durch Produktivitätsverlust)")

def berechne_gesamtkosten(lambda_rate, mu_rate, c, k_server, k_warten):
    """
    Berechnet die Gesamtkosten für ein Warteschlangensystem.
    """
    if c == 1:
        ergebnis = mm1_modell(lambda_rate, mu_rate)
    else:
        ergebnis = mmc_modell(lambda_rate, mu_rate, c)
    
    if 'Fehler' in ergebnis:
        return None
    
    kosten_s = c * k_server
    kosten_w = ergebnis['L'] * k_warten
    kosten_gesamt = kosten_s + kosten_w
    
    return {
        'Server': c,
        'Serverkosten': kosten_s,
        'Wartekosten': kosten_w,
        'Gesamtkosten': kosten_gesamt,
        'L': ergebnis['L'],
        'Wq_min': ergebnis['Wq'] * 60
    }

# Kostenvergleich für verschiedene Serveranzahlen
print("\n📊 KOSTENVERGLEICH:")
print("-" * 70)

kosten_ergebnisse = []
for c in range(1, 6):
    result = berechne_gesamtkosten(lambda_rate, mu_rate, c, kosten_server, kosten_warten)
    if result:
        kosten_ergebnisse.append(result)

df_kosten = pd.DataFrame(kosten_ergebnisse)
df_kosten_display = df_kosten.copy()
df_kosten_display['Serverkosten'] = df_kosten_display['Serverkosten'].apply(lambda x: f"{x:.2f} €")
df_kosten_display['Wartekosten'] = df_kosten_display['Wartekosten'].apply(lambda x: f"{x:.2f} €")
df_kosten_display['Gesamtkosten'] = df_kosten_display['Gesamtkosten'].apply(lambda x: f"{x:.2f} €")
df_kosten_display['Wq_min'] = df_kosten_display['Wq_min'].apply(lambda x: f"{x:.1f} min")
df_kosten_display['L'] = df_kosten_display['L'].apply(lambda x: f"{x:.2f}")

print(df_kosten_display.to_string(index=False))

# Optimum bestimmen
optimum_idx = df_kosten['Gesamtkosten'].idxmin()
optimum = df_kosten.loc[optimum_idx]

print(f"\n🎯 OPTIMUM: {int(optimum['Server'])} Ausgabestelle(n)")
print(f"   Gesamtkosten: {optimum['Gesamtkosten']:.2f} €/Stunde")
print(f"   Wartezeit: {optimum['Wq_min']:.1f} Minuten")

In [ ]:
# ===================================================================
# Visualisierung der Kostenanalyse
# ===================================================================

def plot_kostenanalyse(df_kosten):
    """
    Visualisiert die Kostenkomponenten und das Optimum.
    """
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    server = df_kosten['Server'].values
    
    # Plot 1: Kostenkomponenten
    width = 0.35
    x = np.arange(len(server))
    
    bars1 = ax1.bar(x - width/2, df_kosten['Serverkosten'], width, 
                   label='Serverkosten', color='steelblue')
    bars2 = ax1.bar(x + width/2, df_kosten['Wartekosten'], width, 
                   label='Wartekosten', color='coral')
    
    ax1.plot(x, df_kosten['Gesamtkosten'], 'go-', linewidth=2, markersize=10,
            label='Gesamtkosten')
    
    # Optimum markieren
    opt_idx = df_kosten['Gesamtkosten'].idxmin()
    opt_server = df_kosten.loc[opt_idx, 'Server']
    opt_kosten = df_kosten.loc[opt_idx, 'Gesamtkosten']
    ax1.scatter([opt_server - 1], [opt_kosten], s=200, c='red', zorder=5, 
               marker='*', label=f'Optimum: {int(opt_server)} Server')
    
    ax1.set_xlabel('Anzahl Ausgabestellen', fontsize=11)
    ax1.set_ylabel('Kosten (€/Stunde)', fontsize=11)
    ax1.set_title('Kostenoptimierung Werkzeugausgabe', fontsize=12, fontweight='bold')
    ax1.set_xticks(x)
    ax1.set_xticklabels(server)
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Plot 2: Trade-off Kurve
    ax2.plot(df_kosten['Serverkosten'], df_kosten['Wartekosten'], 'b-o', 
            linewidth=2, markersize=10)
    
    for i, row in df_kosten.iterrows():
        ax2.annotate(f"{int(row['Server'])} Server", 
                    xy=(row['Serverkosten'], row['Wartekosten']),
                    xytext=(5, 5), textcoords='offset points', fontsize=10)
    
    # Optimum markieren
    ax2.scatter([df_kosten.loc[opt_idx, 'Serverkosten']], 
               [df_kosten.loc[opt_idx, 'Wartekosten']], 
               s=200, c='red', zorder=5, marker='*')
    
    ax2.set_xlabel('Serverkosten (€/Stunde)', fontsize=11)
    ax2.set_ylabel('Wartekosten (€/Stunde)', fontsize=11)
    ax2.set_title('Trade-off: Server- vs. Wartekosten', fontsize=12, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_kostenanalyse(df_kosten)

---

## 6. Sensitivitätsanalyse

Die optimale Serveranzahl hängt von den **Kostenparametern** ab. Was passiert, wenn sich diese ändern?

In [ ]:
# ===================================================================
# Sensitivitätsanalyse: Einfluss der Wartekosten
# ===================================================================

def sensitivitaet_wartekosten(lambda_rate, mu_rate, k_server, k_warten_range):
    """
    Analysiert, wie sich das Optimum bei verschiedenen Wartekosten ändert.
    """
    
    ergebnisse = []
    
    for k_w in k_warten_range:
        beste_server = None
        beste_kosten = float('inf')
        
        for c in range(1, 6):
            result = berechne_gesamtkosten(lambda_rate, mu_rate, c, k_server, k_w)
            if result and result['Gesamtkosten'] < beste_kosten:
                beste_kosten = result['Gesamtkosten']
                beste_server = c
        
        ergebnisse.append({
            'Wartekosten': k_w,
            'Optimale_Server': beste_server,
            'Gesamtkosten': beste_kosten
        })
    
    return pd.DataFrame(ergebnisse)

print("🔍 SENSITIVITÄTSANALYSE: Wartekosten")
print("=" * 60)

k_warten_range = np.arange(10, 101, 10)
df_sensitivitaet = sensitivitaet_wartekosten(lambda_rate, mu_rate, kosten_server, k_warten_range)

# Visualisierung
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Optimale Serveranzahl
ax1.step(df_sensitivitaet['Wartekosten'], df_sensitivitaet['Optimale_Server'], 
        where='mid', linewidth=2, color='steelblue')
ax1.scatter(df_sensitivitaet['Wartekosten'], df_sensitivitaet['Optimale_Server'], 
           s=50, color='steelblue', zorder=5)
ax1.axvline(x=kosten_warten, color='red', linestyle='--', 
           label=f'Aktuell: {kosten_warten} €/h')
ax1.set_xlabel('Wartekosten (€/Stunde pro Arbeiter)', fontsize=11)
ax1.set_ylabel('Optimale Anzahl Server', fontsize=11)
ax1.set_title('Optimale Serveranzahl vs. Wartekosten', fontsize=12, fontweight='bold')
ax1.set_yticks(range(1, 6))
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Gesamtkosten
ax2.plot(df_sensitivitaet['Wartekosten'], df_sensitivitaet['Gesamtkosten'], 
        'g-o', linewidth=2)
ax2.axvline(x=kosten_warten, color='red', linestyle='--', 
           label=f'Aktuell: {kosten_warten} €/h')
ax2.set_xlabel('Wartekosten (€/Stunde pro Arbeiter)', fontsize=11)
ax2.set_ylabel('Minimale Gesamtkosten (€/Stunde)', fontsize=11)
ax2.set_title('Minimale Gesamtkosten vs. Wartekosten', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Ergebnistabelle:")
print(df_sensitivitaet.to_string(index=False))

print("\n💡 ERKENNTNIS:")
print("   Bei höheren Wartekosten (z.B. hochqualifizierte Mitarbeiter)")
print("   lohnen sich mehr Ausgabestellen!")

---

## 7. Simulation einer Warteschlange

### Warum Simulation?

Analytische Modelle wie M/M/1 und M/M/c basieren auf **Annahmen** (Poisson-Ankünfte, exponentielle Bedienzeiten). Simulationen ermöglichen:

- **Validierung** der analytischen Ergebnisse
- Analyse **komplexerer Systeme** (z.B. mit Prioritäten)
- Berücksichtigung **realistischerer Verteilungen**
- Untersuchung der **Variabilität** (nicht nur Mittelwerte)

In [ ]:
# ===================================================================
# Ereignisorientierte Simulation einer M/M/1-Warteschlange
# ===================================================================

class MM1Simulation:
    """
    Ereignisorientierte Simulation einer M/M/1-Warteschlange.
    """
    
    def __init__(self, lambda_rate, mu_rate):
        """
        Initialisiert die Simulation.
        
        Parameter:
        ----------
        lambda_rate : float
            Ankunftsrate (Kunden pro Zeiteinheit)
        mu_rate : float
            Bedienrate (Kunden pro Zeiteinheit)
        """
        self.lambda_rate = lambda_rate
        self.mu_rate = mu_rate
    
    def simuliere(self, simulationszeit, seed=42):
        """
        Führt die Simulation durch.
        
        Parameter:
        ----------
        simulationszeit : float
            Dauer der Simulation in Zeiteinheiten
        seed : int
            Seed für Reproduzierbarkeit
            
        Returns:
        --------
        dict : Simulationsergebnisse
        """
        np.random.seed(seed)
        
        # Zustandsvariablen
        zeit = 0
        server_frei_zeit = 0  # Zeitpunkt, wann Server wieder frei wird
        warteschlange = deque()
        
        # Statistiken
        kunden_daten = []
        
        # Erste Ankunft generieren
        naechste_ankunft = np.random.exponential(1/self.lambda_rate)
        
        while zeit < simulationszeit:
            # Entscheide: Ankunft oder Bedienungsende?
            if naechste_ankunft <= server_frei_zeit or len(warteschlange) == 0:
                # Nächstes Ereignis ist eine Ankunft
                zeit = naechste_ankunft
                
                if zeit >= simulationszeit:
                    break
                
                # Kunde erstellen
                kunde = {
                    'ankunft': zeit,
                    'bedienung_start': None,
                    'bedienung_ende': None
                }
                
                # Prüfen ob Server frei
                if server_frei_zeit <= zeit:
                    # Direkt bedienen
                    bedienzeit = np.random.exponential(1/self.mu_rate)
                    kunde['bedienung_start'] = zeit
                    kunde['bedienung_ende'] = zeit + bedienzeit
                    server_frei_zeit = zeit + bedienzeit
                    kunden_daten.append(kunde)
                else:
                    # In Warteschlange einreihen
                    warteschlange.append(kunde)
                
                # Nächste Ankunft planen
                naechste_ankunft = zeit + np.random.exponential(1/self.lambda_rate)
                
            else:
                # Nächstes Ereignis ist ein Bedienungsende
                zeit = server_frei_zeit
                
                # Nächsten Kunden aus Warteschlange nehmen
                if warteschlange:
                    kunde = warteschlange.popleft()
                    bedienzeit = np.random.exponential(1/self.mu_rate)
                    kunde['bedienung_start'] = zeit
                    kunde['bedienung_ende'] = zeit + bedienzeit
                    server_frei_zeit = zeit + bedienzeit
                    kunden_daten.append(kunde)
        
        # Auswertung
        return self._auswertung(kunden_daten)
    
    def _auswertung(self, kunden_daten):
        """
        Wertet die Simulationsergebnisse aus.
        """
        if not kunden_daten:
            return {'Fehler': 'Keine Kunden simuliert'}
        
        # Nur abgeschlossene Kunden
        beendet = [k for k in kunden_daten if k['bedienung_ende'] is not None]
        
        wartezeiten = [k['bedienung_start'] - k['ankunft'] for k in beendet]
        systemzeiten = [k['bedienung_ende'] - k['ankunft'] for k in beendet]
        bedienzeiten = [k['bedienung_ende'] - k['bedienung_start'] for k in beendet]
        
        return {
            'n_kunden': len(beendet),
            'wartezeiten': wartezeiten,
            'systemzeiten': systemzeiten,
            'bedienzeiten': bedienzeiten,
            'Wq_mean': np.mean(wartezeiten),
            'Wq_std': np.std(wartezeiten),
            'W_mean': np.mean(systemzeiten),
            'W_std': np.std(systemzeiten),
            'Wq_max': np.max(wartezeiten),
            'W_max': np.max(systemzeiten)
        }

# Simulation durchführen
print("🎮 SIMULATION DER WERKZEUGAUSGABE")
print("=" * 60)

sim = MM1Simulation(lambda_rate, mu_rate)
sim_ergebnis = sim.simuliere(simulationszeit=100, seed=42)  # 100 Stunden

print(f"\nSimulationsparameter:")
print(f"   Simulationsdauer: 100 Stunden")
print(f"   Simulierte Kunden: {sim_ergebnis['n_kunden']}")

print(f"\n📊 VERGLEICH: Theorie vs. Simulation")
print("-" * 50)
print(f"{'Kennzahl':<30} {'Theorie':>12} {'Simulation':>12}")
print("-" * 50)
print(f"{'Mittlere Wartezeit (min)':<30} {ergebnis_mm1['Wq']*60:>12.2f} {sim_ergebnis['Wq_mean']*60:>12.2f}")
print(f"{'Mittlere Systemzeit (min)':<30} {ergebnis_mm1['W']*60:>12.2f} {sim_ergebnis['W_mean']*60:>12.2f}")
print(f"{'Max. Wartezeit (min)':<30} {'—':>12} {sim_ergebnis['Wq_max']*60:>12.2f}")
print(f"{'Standardabw. Wartezeit (min)':<30} {'—':>12} {sim_ergebnis['Wq_std']*60:>12.2f}")

# Abweichung berechnen
abw_Wq = abs(sim_ergebnis['Wq_mean'] - ergebnis_mm1['Wq']) / ergebnis_mm1['Wq'] * 100
print(f"\n   Abweichung Wartezeit: {abw_Wq:.1f}%")

In [ ]:
# ===================================================================
# Visualisierung der Simulationsergebnisse
# ===================================================================

def plot_simulation(sim_ergebnis, theorie_Wq):
    """
    Visualisiert die Verteilung der Warte- und Systemzeiten aus der Simulation.
    """
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    wartezeiten_min = [w * 60 for w in sim_ergebnis['wartezeiten']]
    systemzeiten_min = [s * 60 for s in sim_ergebnis['systemzeiten']]
    
    # 1. Histogramm Wartezeiten
    ax1 = axes[0, 0]
    ax1.hist(wartezeiten_min, bins=30, density=True, alpha=0.7, 
            color='steelblue', edgecolor='black', label='Simulation')
    
    # Theoretische Verteilung (exponentiell für M/M/1)
    x_theo = np.linspace(0, max(wartezeiten_min), 100)
    # Wartezeit ist mit Wahrscheinlichkeit ρ exponentialverteilt
    ax1.axvline(x=theorie_Wq*60, color='red', linestyle='--', linewidth=2,
               label=f'Theorie: {theorie_Wq*60:.1f} min')
    ax1.axvline(x=np.mean(wartezeiten_min), color='green', linestyle='-', linewidth=2,
               label=f'Simulation: {np.mean(wartezeiten_min):.1f} min')
    
    ax1.set_xlabel('Wartezeit (Minuten)', fontsize=11)
    ax1.set_ylabel('Relative Häufigkeit', fontsize=11)
    ax1.set_title('Verteilung der Wartezeiten', fontsize=12, fontweight='bold')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Histogramm Systemzeiten
    ax2 = axes[0, 1]
    ax2.hist(systemzeiten_min, bins=30, density=True, alpha=0.7, 
            color='coral', edgecolor='black')
    ax2.axvline(x=np.mean(systemzeiten_min), color='green', linestyle='-', linewidth=2,
               label=f'Mittelwert: {np.mean(systemzeiten_min):.1f} min')
    ax2.set_xlabel('Systemzeit (Minuten)', fontsize=11)
    ax2.set_ylabel('Relative Häufigkeit', fontsize=11)
    ax2.set_title('Verteilung der Systemzeiten', fontsize=12, fontweight='bold')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Boxplot
    ax3 = axes[1, 0]
    bp = ax3.boxplot([wartezeiten_min, systemzeiten_min], 
                    labels=['Wartezeit', 'Systemzeit'],
                    patch_artist=True)
    bp['boxes'][0].set_facecolor('steelblue')
    bp['boxes'][1].set_facecolor('coral')
    ax3.set_ylabel('Zeit (Minuten)', fontsize=11)
    ax3.set_title('Boxplot der Zeiten', fontsize=12, fontweight='bold')
    ax3.grid(True, alpha=0.3)
    
    # 4. Zeitverlauf der Wartezeiten (erste 200 Kunden)
    ax4 = axes[1, 1]
    n_plot = min(200, len(wartezeiten_min))
    ax4.plot(range(n_plot), wartezeiten_min[:n_plot], 'b-', alpha=0.7, linewidth=0.5)
    ax4.axhline(y=np.mean(wartezeiten_min), color='red', linestyle='--', linewidth=2,
               label=f'Mittelwert: {np.mean(wartezeiten_min):.1f} min')
    ax4.set_xlabel('Kunde Nr.', fontsize=11)
    ax4.set_ylabel('Wartezeit (Minuten)', fontsize=11)
    ax4.set_title('Zeitverlauf der Wartezeiten', fontsize=12, fontweight='bold')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_simulation(sim_ergebnis, ergebnis_mm1['Wq'])

print("\n💡 ERKENNTNISSE AUS DER SIMULATION:")
print("   • Die theoretischen Werte werden gut approximiert")
print("   • Die Wartezeiten schwanken stark (hohe Variabilität)")
print(f"   • Einige Kunden warten bis zu {sim_ergebnis['Wq_max']*60:.0f} Minuten!")
print("   • Die Verteilung ist rechtsschief (viele kurze, wenige lange Wartezeiten)")

---

## 8. Aufgaben für Studierende

Bearbeiten Sie die folgenden Aufgaben, um Ihr Verständnis der Warteschlangentheorie zu vertiefen.

### ✏️ Aufgabe 1: Neues Szenario - CNC-Bearbeitungszentrum

Ein CNC-Bearbeitungszentrum erhält Werkstücke zur Bearbeitung:

| Parameter | Wert |
|-----------|------|
| Ankunftsrate | 12 Werkstücke/Stunde |
| Bearbeitungsrate pro Maschine | 8 Werkstücke/Stunde |
| Maschinenkosten | 50 €/Stunde |
| Wartekosten pro Werkstück | 20 €/Stunde |

**Fragen:**
1. Ist ein System mit nur 1 Maschine stabil? Warum (nicht)?
2. Berechnen Sie die Kennzahlen für 2 und 3 Maschinen.
3. Wie viele Maschinen sind kostenoptimal?

In [ ]:
# Ihre Lösung für Aufgabe 1:
# ===========================

# TODO: Definieren Sie die Parameter
# TODO: Prüfen Sie die Stabilität für c=1
# TODO: Berechnen Sie Kennzahlen für c=2 und c=3
# TODO: Führen Sie eine Kostenanalyse durch



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 1 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 1: CNC-Bearbeitungszentrum")
print("=" * 65)

# Parameter
lambda_cnc = 12  # Werkstücke/Stunde
mu_cnc = 8       # Werkstücke/Stunde pro Maschine
k_maschine = 50  # €/Stunde
k_warten_cnc = 20 # €/Stunde pro Werkstück

print("\n1️⃣ STABILITÄTSPRÜFUNG für c = 1:")
print("-" * 40)
rho_1 = lambda_cnc / mu_cnc
print(f"   ρ = λ/μ = {lambda_cnc}/{mu_cnc} = {rho_1:.2f}")

if rho_1 >= 1:
    print(f"   ❌ System ist INSTABIL (ρ = {rho_1:.2f} ≥ 1)")
    print(f"   Die Ankunftsrate ({lambda_cnc}/h) übersteigt die Kapazität ({mu_cnc}/h)!")
    print(f"   → Mindestens 2 Maschinen erforderlich.")
else:
    print(f"   ✅ System ist stabil")

print("\n2️⃣ KENNZAHLEN für c = 2 und c = 3:")
print("-" * 40)

for c in [2, 3, 4]:
    erg = mmc_modell(lambda_cnc, mu_cnc, c)
    if 'Fehler' not in erg:
        print(f"\n   {c} Maschine(n):")
        print(f"      Auslastung pro Maschine: {erg['rho']*100:.1f}%")
        print(f"      Mittlere Warteschlangenlänge: {erg['Lq']:.2f} Werkstücke")
        print(f"      Mittlere Wartezeit: {erg['Wq']*60:.1f} Minuten")
        print(f"      Mittlere Durchlaufzeit: {erg['W']*60:.1f} Minuten")
    else:
        print(f"\n   {c} Maschine(n): {erg['Fehler']}")

print("\n3️⃣ KOSTENANALYSE:")
print("-" * 40)

print(f"\n{'Maschinen':>10} {'Maschinenkosten':>18} {'Wartekosten':>15} {'Gesamt':>12}")
print("-" * 60)

beste_kosten = float('inf')
beste_c = None

for c in range(2, 6):
    erg = mmc_modell(lambda_cnc, mu_cnc, c)
    if 'Fehler' not in erg:
        k_m = c * k_maschine
        k_w = erg['L'] * k_warten_cnc
        k_ges = k_m + k_w
        
        if k_ges < beste_kosten:
            beste_kosten = k_ges
            beste_c = c
        
        print(f"{c:>10} {k_m:>15.2f} € {k_w:>14.2f} € {k_ges:>11.2f} €")

print("-" * 60)
print(f"\n🎯 OPTIMUM: {beste_c} Maschinen mit Gesamtkosten von {beste_kosten:.2f} €/Stunde")

### ✏️ Aufgabe 2: Simulation und Vergleich

Erweitern Sie die Simulation für das Werkzeugausgabe-Beispiel:

1. Führen Sie die Simulation mit verschiedenen Seeds durch (z.B. 10 Durchläufe)
2. Berechnen Sie das 95%-Konfidenzintervall für die mittlere Wartezeit
3. Vergleichen Sie das Intervall mit dem theoretischen Wert

*Hinweis: Konfidenzintervall = Mittelwert ± 1.96 × Standardfehler*

In [ ]:
# Ihre Lösung für Aufgabe 2:
# ===========================

# TODO: Führen Sie mehrere Simulationsdurchläufe durch
# TODO: Berechnen Sie Mittelwert und Standardfehler
# TODO: Bestimmen Sie das 95%-Konfidenzintervall



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 2 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 2: Simulation mit Konfidenzintervall")
print("=" * 65)

# Mehrere Simulationsdurchläufe
n_durchlaeufe = 20
simulationszeit = 200  # Stunden

wq_werte = []

print(f"\nDurchführung von {n_durchlaeufe} Simulationen à {simulationszeit} Stunden...")

sim = MM1Simulation(lambda_rate, mu_rate)

for seed in range(n_durchlaeufe):
    ergebnis = sim.simuliere(simulationszeit=simulationszeit, seed=seed*100)
    wq_werte.append(ergebnis['Wq_mean'] * 60)  # in Minuten

# Statistik berechnen
wq_mean = np.mean(wq_werte)
wq_std = np.std(wq_werte, ddof=1)  # Stichproben-Standardabweichung
standardfehler = wq_std / np.sqrt(n_durchlaeufe)

# 95% Konfidenzintervall
ki_unten = wq_mean - 1.96 * standardfehler
ki_oben = wq_mean + 1.96 * standardfehler

# Theoretischer Wert
wq_theorie = ergebnis_mm1['Wq'] * 60

print(f"\n📊 ERGEBNISSE:")
print("-" * 50)
print(f"   Anzahl Durchläufe:        {n_durchlaeufe}")
print(f"   Mittelwert Wq:            {wq_mean:.2f} Minuten")
print(f"   Standardabweichung:       {wq_std:.2f} Minuten")
print(f"   Standardfehler:           {standardfehler:.3f} Minuten")
print(f"\n   95%-Konfidenzintervall:   [{ki_unten:.2f}, {ki_oben:.2f}] Minuten")
print(f"   Theoretischer Wert:       {wq_theorie:.2f} Minuten")

# Prüfen ob theoretischer Wert im KI liegt
if ki_unten <= wq_theorie <= ki_oben:
    print(f"\n   ✅ Der theoretische Wert liegt im Konfidenzintervall!")
else:
    print(f"\n   ⚠️ Der theoretische Wert liegt NICHT im Konfidenzintervall.")

# Visualisierung
plt.figure(figsize=(10, 5))
plt.hist(wq_werte, bins=10, density=True, alpha=0.7, color='steelblue', edgecolor='black')
plt.axvline(x=wq_theorie, color='red', linestyle='--', linewidth=2, label=f'Theorie: {wq_theorie:.2f} min')
plt.axvline(x=wq_mean, color='green', linestyle='-', linewidth=2, label=f'Sim. Mittelwert: {wq_mean:.2f} min')
plt.axvspan(ki_unten, ki_oben, alpha=0.3, color='green', label='95% KI')
plt.xlabel('Mittlere Wartezeit (Minuten)', fontsize=11)
plt.ylabel('Relative Häufigkeit', fontsize=11)
plt.title('Verteilung der mittleren Wartezeit über mehrere Simulationen', fontsize=12, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### ✏️ Aufgabe 3: Einfluss der Variabilität

Die Warteschlangentheorie zeigt: **Höhere Variabilität führt zu längeren Wartezeiten!**

Vergleichen Sie für das Werkzeugausgabe-Beispiel:
- **Szenario A**: Exponentielle Bedienzeit (wie bisher), Variationskoeffizient CV = 1
- **Szenario B**: Konstante Bedienzeit (M/D/1), Variationskoeffizient CV = 0

Für M/D/1 gilt: $L_q = \frac{\rho^2}{2(1-\rho)}$ (halb so viel wie M/M/1!)

**Fragen:**
1. Berechnen Sie $L_q$ und $W_q$ für beide Szenarien
2. Um wie viel Prozent sinkt die Wartezeit bei konstanter Bedienzeit?

In [ ]:
# Ihre Lösung für Aufgabe 3:
# ===========================

# TODO: Berechnen Sie Lq und Wq für M/M/1 (exponentiell)
# TODO: Berechnen Sie Lq und Wq für M/D/1 (konstant)
# TODO: Vergleichen Sie die Ergebnisse



In [ ]:
# ===================================================================
# LÖSUNG Aufgabe 3 (zur Selbstkontrolle)
# ===================================================================

print("📝 LÖSUNG AUFGABE 3: Einfluss der Variabilität")
print("=" * 65)

def md1_modell(lambda_rate, mu_rate):
    """
    M/D/1 Modell: Poisson-Ankünfte, DETERMINISTISCHE (konstante) Bedienzeit
    """
    if lambda_rate >= mu_rate:
        return {'Fehler': 'System ist instabil'}
    
    rho = lambda_rate / mu_rate
    
    # Für M/D/1: Lq ist halb so groß wie bei M/M/1!
    Lq = (rho ** 2) / (2 * (1 - rho))
    L = Lq + rho
    Wq = Lq / lambda_rate
    W = Wq + 1/mu_rate
    
    return {
        'rho': rho,
        'Lq': Lq,
        'L': L,
        'Wq': Wq,
        'W': W
    }

# Berechnungen
mm1 = mm1_modell(lambda_rate, mu_rate)
md1 = md1_modell(lambda_rate, mu_rate)

print("\n📊 VERGLEICH: M/M/1 vs. M/D/1")
print("-" * 60)
print(f"{'Kennzahl':<35} {'M/M/1':>12} {'M/D/1':>12}")
print("-" * 60)
print(f"{'Variationskoeffizient Bedienzeit':<35} {'1.00':>12} {'0.00':>12}")
print(f"{'Mittlere Anzahl wartend (Lq)':<35} {mm1['Lq']:>12.3f} {md1['Lq']:>12.3f}")
print(f"{'Mittlere Anzahl im System (L)':<35} {mm1['L']:>12.3f} {md1['L']:>12.3f}")
print(f"{'Mittlere Wartezeit (min)':<35} {mm1['Wq']*60:>12.2f} {md1['Wq']*60:>12.2f}")
print(f"{'Mittlere Systemzeit (min)':<35} {mm1['W']*60:>12.2f} {md1['W']*60:>12.2f}")

# Verbesserung berechnen
verbesserung_Lq = (mm1['Lq'] - md1['Lq']) / mm1['Lq'] * 100
verbesserung_Wq = (mm1['Wq'] - md1['Wq']) / mm1['Wq'] * 100

print("-" * 60)
print(f"{'Reduktion Warteschlangenlänge:':<35} {verbesserung_Lq:>24.1f}%")
print(f"{'Reduktion Wartezeit:':<35} {verbesserung_Wq:>24.1f}%")

print("\n💡 ERKENNTNIS:")
print("   Bei konstanter (deterministischer) Bedienzeit halbiert sich")
print("   die Warteschlangenlänge im Vergleich zu exponentieller Bedienzeit!")
print("\n   → Standardisierung und Taktzeitreduktion in der Produktion")
print("     können Wartezeiten erheblich reduzieren!")

---

## 9. Zusammenfassung und Ausblick

### Was haben wir gelernt?

| Thema | Kernaussage |
|-------|-------------|
| **Kendall-Notation** | A/B/c beschreibt Ankunftsprozess, Bedienprozess und Serveranzahl |
| **M/M/1-Modell** | Grundmodell mit geschlossenen Formeln für alle Kennzahlen |
| **M/M/c-Modell** | Erweiterung auf mehrere Server; Wartezeit sinkt überproportional |
| **Kostenoptimierung** | Trade-off zwischen Server- und Wartekosten |
| **Simulation** | Validierung und Analyse komplexerer Systeme |
| **Variabilität** | Höhere Variabilität = längere Wartezeiten |

### Wichtige Formeln (M/M/1)

| Kennzahl | Formel |
|----------|--------|
| Auslastung | $\rho = \lambda / \mu$ |
| Mittlere Anzahl im System | $L = \rho / (1-\rho)$ |
| Mittlere Wartezeit | $W_q = \rho / (\mu - \lambda)$ |
| Little's Gesetz | $L = \lambda \cdot W$ |

### Grenzen der Warteschlangentheorie

- ❌ Strenge Annahmen (Poisson-Ankünfte, exponentielle Bedienzeiten)
- ❌ Nur **stationäre** Lösungen (keine Anlaufeffekte)
- ❌ Komplexe Systeme (Netzwerke) analytisch schwer lösbar

### Weiterführende Themen

- **Warteschlangennetzwerke** (Jackson-Netzwerke)
- **Prioritätswarteschlangen** (Vorrangregeln)
- **Allgemeine Verteilungen** (G/G/1-Approximationen)
- **Transiente Analyse** (Nicht-stationäres Verhalten)

In [ ]:
# ===================================================================
# Abschluss und Checkliste
# ===================================================================

print("\n" + "="*70)
print("📋 CHECKLISTE - Warteschlangentheorie")
print("="*70)

checkliste = [
    ("Warteschlangensystem erkennen", 
     "Kunden, Server, Ankunftsprozess, Bedienprozess identifizieren"),
    ("Kendall-Notation anwenden", 
     "A/B/c korrekt bestimmen (z.B. M/M/1, M/M/c, M/D/1)"),
    ("Stabilitätsbedingung prüfen", 
     "ρ = λ/(c·μ) < 1 sicherstellen"),
    ("M/M/1-Kennzahlen berechnen", 
     "L, Lq, W, Wq mit geschlossenen Formeln"),
    ("M/M/c-Modell anwenden", 
     "Erlang-C Formel für mehrere Server"),
    ("Kostenoptimierung durchführen", 
     "Trade-off Server- vs. Wartekosten analysieren"),
    ("Simulation einsetzen", 
     "Validierung und Analyse komplexer Systeme")
]

for i, (punkt, beschreibung) in enumerate(checkliste, 1):
    print(f"\n☐ {i}. {punkt}")
    print(f"   → {beschreibung}")

print("\n" + "="*70)
print("🎓 Viel Erfolg bei der weiteren Vertiefung!")
print("   Nächstes Thema: Data Analytics & KI")
print("="*70)